# Artificial Neural Network

### Import libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
tf.__version__

'2.12.0'

## Part 1 - Data Preprocessing

### Import dataset

In [4]:
dataset = pd.read_csv('Churn_Modelling.csv')
x = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [5]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [6]:
print(y)

[1 0 1 ... 1 1 0]


### Encode categorical data

Label encode "Gender" variable

In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
x[:, 2] = le.fit_transform(x[:, 2])

In [ ]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One hot encode "Geography" variable

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder = 'passthrough')
x = np.array(ct.fit_transform(x))

In [ ]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Split dataset into training and testing sets

In [7]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

## Part 2 - Build ANN

### Initialize ANN

In [9]:
ann = tf.keras.models.Sequential()

### Add input layer and first hidden layer

In [10]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Add second hidden layer

In [11]:
ann.add(tf.keras.layers.Dense(units = 6, activation = 'relu'))

### Add output layer

In [12]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

## Part 3 - Train ANN

### Compile ANN

In [13]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Train ANN on training set

In [14]:
ann.fit(x_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 2s 3ms/step - loss: 0.5154 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4641 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4456 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4346 - accuracy: 0.8025
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4269 - accuracy: 0.8141
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4201 - accuracy: 0.8200
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4132 - accuracy: 0.8242
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4040 - accuracy: 0.8306
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3934 - accuracy: 0.8401
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3828 - accura

## Part 4 - Make predictions and evaluate model

### Predict result of a single observation

**Customer profile:**

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Has credit card?: Yes

Active member?: Yes

Estimated Salary: \$ 50000

**Probability of leaving**

In [17]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))

1/1 [==============================] - 0s 39ms/step
[[0.03447769]]


### Compare test set predictions and results

In [21]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.4)
print(np.concatenate((y_pred.reshape(len(y_pred),1),y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Confusion Matrix

In [22]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1468  127]
 [ 149  256]]


0.862